In [3]:
# task_3_hypothesis_testing/calculate_metrics.py
import pandas as pd
import numpy as np
from pathlib import Path

In [4]:
# Load data (assuming DVC-tracked data)
data =  pd.read_csv(r"C:\10x AIMastery\Insurance-risk-analytics\data\insurance_data.csv.dvc") # Update path as needed

In [5]:
# task_3_hypothesis_testing/hypothesis_testing.py
from scipy.stats import f_oneway, chi2_contingency

In [17]:
# Create required columns if they don't exist
if 'HasClaim' not in data.columns:
    data['HasClaim'] = data['TotalClaims'] > 0
if 'Margin' not in data.columns:
    data['Margin'] = data['TotalPremium'] - data['TotalClaims']

# Verify available columns and data
print("Available columns:", data.columns.tolist())
print("HasClaim unique values:", data['HasClaim'].unique())
print("Margin sample values:", data['Margin'].head())

# ANOVA for Claim Severity across provinces
severity_by_province = [
    data.loc[(data['Province'] == p) & data['HasClaim'], 'TotalClaims'].dropna()
    for p in data['Province'].unique() if data.loc[(data['Province'] == p) & data['HasClaim'], 'TotalClaims'].size > 0
]
p_value_severity = None  # Initialize to avoid NameError
if len(severity_by_province) >= 2:  # Ensure at least 2 groups for ANOVA
    f_stat, p_value_severity = f_oneway(*severity_by_province)
    print(f"ANOVA for Claim Severity across Provinces: p-value = {p_value_severity:.4f}")
else:
    print("Insufficient data for ANOVA on Claim Severity (fewer than 2 provinces with claims).")

# Interpret result
severity_result = (
    "Reject H01: Significant differences in Claim Severity across provinces."
    if p_value_severity is not None and p_value_severity < 0.05 and len(severity_by_province) >= 2
    else "Fail to reject H01: No significant differences in Claim Severity."
)
print(severity_result)

# Save results with UTF-8 encoding
with open(r"C:\10x AIMastery\Insurance-risk-analytics\task_3_hypothesis_testing\h01_severity_results.txt", "w", encoding="utf-8") as f:
    f.write(severity_result)

Available columns: ['outs:', 'TotalPremium', 'TotalClaims', 'Province', 'Gender', 'VehicleType', 'RegistrationYear', 'SumInsured', 'CoverType', 'TransactionMonth', 'PostalCode', 'HasClaim', 'Margin']
HasClaim unique values: [False  True]
Margin sample values: 0      NaN
1      NaN
2      NaN
3      NaN
4    900.0
Name: Margin, dtype: float64
Insufficient data for ANOVA on Claim Severity (fewer than 2 provinces with claims).
Fail to reject H01: No significant differences in Claim Severity.


In [19]:
from scipy.stats import ttest_ind
from statsmodels.stats.proportion import proportions_ztest

# Filter for Women and Men
women = data[data['Gender'] == 'Female']
men = data[data['Gender'] == 'Male']

In [20]:
# Z-test for Claim Frequency
count = [women['HasClaim'].sum(), men['HasClaim'].sum()]
nobs = [women['HasClaim'].count(), men['HasClaim'].count()]
z_stat, p_value_freq = proportions_ztest(count, nobs)
print(f"Z-test for Claim Frequency (Women vs. Men): p-value = {p_value_freq:.4f}")

Z-test for Claim Frequency (Women vs. Men): p-value = nan


c:\10x AIMastery\Insurance-risk-analytics\venv\Lib\site-packages\statsmodels\stats\proportion.py:1004: RuntimeWarning: invalid value encountered in divide
  prop = count * 1. / nobs
c:\10x AIMastery\Insurance-risk-analytics\venv\Lib\site-packages\statsmodels\stats\proportion.py:1020: RuntimeWarning: divide by zero encountered in divide
  nobs_fact = np.sum(1. / nobs)
c:\10x AIMastery\Insurance-risk-analytics\venv\Lib\site-packages\statsmodels\stats\proportion.py:1023: RuntimeWarning: invalid value encountered in scalar multiply
  var_ = p_pooled * (1 - p_pooled) * nobs_fact


In [21]:
# T-test for Claim Severity
severity_women = women.loc[women['HasClaim'], 'TotalClaims'].dropna()
severity_men = men.loc[men['HasClaim'], 'TotalClaims'].dropna()
t_stat, p_value_sev = ttest_ind(severity_women, severity_men, equal_var=False)
print(f"T-test for Claim Severity (Women vs. Men): p-value = {p_value_sev:.4f}")

T-test for Claim Severity (Women vs. Men): p-value = nan


c:\10x AIMastery\Insurance-risk-analytics\venv\Lib\site-packages\scipy\_lib\deprecation.py:234: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  return f(*args, **kwargs)


In [22]:
# Interpret results
if p_value_freq < 0.05:
    print("Reject H₀: Significant differences in Claim Frequency between Women and Men.")
else:
    print("Fail to reject H₀: No significant differences in Claim Frequency.")

Fail to reject H₀: No significant differences in Claim Frequency.
